In [1]:
#!/usr/bin/python3.6
from pynq import Xlnk
from pynq import Overlay
from pynq.lib.video import *
from IPython import display
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from scipy import signal
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import time
import sys
import cv2
import numpy as np
import numpy.fft as fft
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import numpy.fft as fft

import serial
import serial.tools.list_ports
######################################################
bitfilename = '/home/xilinx/jupyter_notebooks/demo1101/sys_1115.bit'
psfname = '/home/xilinx/jupyter_notebooks/demo1101/psf.jpg'
imgname = '/home/xilinx/jupyter_notebooks/demo1101/raw.jpg'
graphic_dir = '/home/xilinx/jupyter_notebooks/demo1101/graphic/'
######################################################
iters = 60
image_width = 640
image_height = 480
################### download the overlay #####################
xlnk = Xlnk()
xlnk.xlnk_reset()
overlay = Overlay(bitfilename)
print("bit file  loaded!")
####################Initialize Hardware##############################
overlay.axi_gpio_rgb[3].write(1)#sign as setting up
grad_ip= overlay.FISTA_GRAD_0
trans_ip=overlay.TRANS_FIX_0
vdma=overlay.axi_vdma_0
framemode = VideoMode(image_width,image_height,24)
vdma.readchannel.mode = framemode
vdma.writechannel.mode = framemode
#####print vdma's status
vdma.readchannel.start()
vdma.writechannel.start()
print('0x%x'%vdma.read(0x34))
print('0x%x'%vdma.read(0x04))
print("#################################")

bit file  loaded!
0x40000
0x10000
#################################


In [2]:
################################################
def manual_arm():
    x=10
    y=10
    inv_t = 0.005
    while (overlay.axi_gpio_sw0.read()):
        time.sleep(0.03)
        keydown=overlay.axi_gpio_btn4.read()
        if(keydown==8):#OFF(EXIT)
            list4[4]=x
            for i in list4:
                ser.write(chr(i).encode("utf-8"))
        elif(keydown==1):#OFF(EXIT)
            list5[4]=x
            for i in list5:
                ser.write(chr(i).encode("utf-8"))
        elif(keydown==4):#OFF(EXIT)
            list1[4]=y
            for i in list1:
                ser.write(chr(i).encode("utf-8"))
        elif(keydown==2):#OFF(EXIT)
            list2[4]=y
            for i in list2:
                ser.write(chr(i).encode("utf-8"))
    return 
def auto_trace():
    vdma.readchannel.stop()
    vdma.readchannel.start()
    vdma.writechannel.start()
    time.sleep(0.5)
    start=time.time()
    run_4ever = 1
    #定义结构元素
    #kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5, 5))
    #kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(50, 50))
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(3, 3))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(7, 7))
    while (run_4ever == 1):
        #time.sleep(0.05)
        keydown=overlay.axi_gpio_btn4.read()
        if(keydown==1):#OFF(EXIT)
            break
        if (time.time()-start)>60:
            for i in list7:
                ser.write(chr(i).encode("utf-8"))
            break
        frame = vdma.readchannel.readframe()
        res_frame=cv2.resize(frame,(160,120))###########减小了输入
        start_t = time.time()
        psfly = cv2.cvtColor(res_frame, cv2.COLOR_BGR2GRAY)
        ret, binary = cv2.threshold(psfly,5,255,cv2.THRESH_BINARY)
        #ret, binary = cv2.threshold(psfly, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)#大律法,
        #blured=cv2.medianBlur(binary,19)
        #开闭运算，先开运算去除背景噪声，再继续闭运算填充目标内的孔洞
        opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel1)
        closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel2)
        #开闭运算，先开运算去除背景噪声，再继续闭运算填充目标内的孔洞
        opened1 = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel1)
        closed1 = cv2.morphologyEx(opened1, cv2.MORPH_CLOSE, kernel2)
        opened2 = cv2.morphologyEx(closed1, cv2.MORPH_OPEN, kernel1)
        closed2 = cv2.morphologyEx(opened2, cv2.MORPH_CLOSE, kernel2)
        ret, binary2 = cv2.threshold(closed2,5,255,cv2.THRESH_BINARY)
        binary2 = cv2.resize(binary2,(640,480))
        _,contours,hierarchy = cv2.findContours(binary2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        if len(contours)==0:
            continue
        else:
            M = cv2.moments(contours[0])  # 计算第一条轮廓的各阶矩,字典形式
            #center_x = int(M["m10"] / M["m00"])
            #center_y = int(M["m01"] / M["m00"])
            center_x = int(M["m10"] / M["m00"])
            center_y = int(M["m01"] / M["m00"])

            x=abs(center_x-320)
            if x>200:
                x=200
            y=abs(center_y-240)
            if y>150:
                y=150
            #绘制结果
            cv2.drawContours(frame,contours, 0, (0,0,255),4)
            cv2.circle(frame, (center_x, center_y), 9,(0,0,255), -1)#绘制中心点
            print(time.time()-start_t)
            inv_t = 0.014
            print(center_x,center_y)
            if center_x<300 or center_x>340:
                vdma.writechannel.writeframe(frame)
                if center_x<300:
                    list4[4]=x
                    for i in list4:
                        ser.write(chr(i).encode("utf-8"))
                elif center_x>340:
                    list5[4]=x
                    for i in list5:
                        ser.write(chr(i).encode("utf-8"))
                time.sleep(inv_t * x)
            elif center_y<220 or center_y>260:
                vdma.writechannel.writeframe(frame)
                if center_y<220:
                    list1[4]=y
                    for i in list1:
                        ser.write(chr(i).encode("utf-8"))
                elif center_y>260:
                    list2[4]=y
                    for i in list2:
                        ser.write(chr(i).encode("utf-8"))
                time.sleep(inv_t * y)
            elif (center_x>=300 and center_x<=340 and center_y>=220 and center_y<=260):
                for i in list7:
                    ser.write(chr(i).encode("utf-8"))
                break
    time.sleep(0.8)
    frame = vdma.readchannel.readframe()
    cv2.drawContours(frame,contours, 0, (0,0,255),4)
    cv2.circle(frame, (center_x, center_y), 9,(0,0,255), -1)#绘制中心点
    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,'Center!',(20,70),font,1,(0,0,255),2)
    vdma.writechannel.writeframe(frame)
def capture_raw(mode):
    vdma.readchannel.stop()
    vdma.readchannel.start()
    vdma.writechannel.start()
    time.sleep(0.5)
    frame = vdma.readchannel.readframe()
    r,g,b=cv2.split(frame)
    om=cv2.merge([b,r,g])
    tip_text = 'Press button[2] again to save.'
    font=cv2.FONT_HERSHEY_PLAIN#cv2.FONT_HERSHEY_SIMPLEX
    blank = np.zeros((240,320))
    if(mode==1):
        img_save_name = '/home/xilinx/jupyter_notebooks/demo1101/psf.jpg'
        dis_text = 'PSF captured!'
        cv2.putText(frame,dis_text,(20,70),font,3,(0,0,255),2)
        vdma.writechannel.writeframe(frame)
        overlay.axi_gpio_rgb[0].write(1)
        
        gray_om = cv2.cvtColor(om, cv2.COLOR_RGB2GRAY)
        gray_om = gray_om[120:360,160:480]
        gray_om = cv2.resize(gray_om, (160, 120))
        gray_om = np.array(gray_om, dtype='float32')
        print('start correlate!')
        corr = signal.correlate2d(gray_om, gray_om, boundary='symm', mode='same')
        overlay.axi_gpio_rgb[0].write(0)
        print('over correlate!')
        psf_norm = cv2.normalize(gray_om,None,0,255,cv2.NORM_MINMAX)
        corr_norm = cv2.normalize(corr,None,0,255,cv2.NORM_MINMAX)
        hmerge = np.hstack((psf_norm, corr_norm)) #水平拼接
        blank[60:180,:]=hmerge
        trans_frame(blank,dis_text='Autocorrelation     (btn[2] to save)')

    else:
        img_save_name = '/home/xilinx/jupyter_notebooks/demo1101/raw.jpg'
        dis_text = 'RAW captured!'
        
        cv2.putText(frame,dis_text,(20,70),font,3,(0,0,255),2)
        cv2.putText(frame,tip_text,(160,420),font,1,(0,0,255),1)
        vdma.writechannel.writeframe(frame)
    #plt.imsave(img_save_name,om)
    

    start = time.time()
    keydown=0
    while(time.time()-start<10):
        keydown=overlay.axi_gpio_btn4.read()
        if(keydown == 4):
            plt.imsave(img_save_name,om)
            break
        elif(keydown==1):
            break
        time.sleep(0.1)
def recover_stream():
    vdma.readchannel.stop()
    vdma.readchannel.start()
    vdma.writechannel.start()
    vdma.readchannel.tie(vdma.writechannel)
def blink_led4(num):
    for i in range(0,num):
        time.sleep(0.25)
        for led in range(0,4):
            overlay.axi_gpio_led4[led].write(0)
        time.sleep(0.25)
        for led in range(0,4):
            overlay.axi_gpio_led4[led].write(1)
def water_led4(num):
    for led in range(0,4):
        overlay.axi_gpio_led4[led].write(0)
    for i in range(0,num):
        for led in range(0,4):
            overlay.axi_gpio_led4[led].write(1)
            time.sleep(0.25)
        for led in range(0,4):
            overlay.axi_gpio_led4[led].write(0)
            time.sleep(0.25)

In [ ]:
#########
DDR_b_axi = xlnk.cma_array(shape=(120,160), dtype=np.float32)
DDR_vk_axi = xlnk.cma_array(shape=(256,512), dtype=np.float32)
DDR_grad_out = xlnk.cma_array(shape=(256,512), dtype=np.float32)

DDR_H_axi_re = xlnk.cma_array(shape=(256,512), dtype=np.uint32)
DDR_H_axi_im = xlnk.cma_array(shape=(256,512), dtype=np.uint32)
vk_mid_axi_re = xlnk.cma_array(shape=(256,512), dtype=np.uint32)
vk_mid_axi_im = xlnk.cma_array(shape=(256,512), dtype=np.uint32)
print("FISTA memory allocated!")
H_float_re=xlnk.cma_array(shape=(256,512), dtype=np.float32)
H_float_im=xlnk.cma_array(shape=(256,512), dtype=np.float32)
print("TRANS memory allocated!")

grad_ip.write(0x10, DDR_b_axi.physical_address)

grad_ip.write(0x18, DDR_H_axi_re.physical_address)
grad_ip.write(0x20, DDR_H_axi_im.physical_address)
grad_ip.write(0x28, DDR_vk_axi.physical_address)

grad_ip.write(0x30, vk_mid_axi_re.physical_address)
grad_ip.write(0x38, vk_mid_axi_im.physical_address)

grad_ip.write(0x40, DDR_grad_out.physical_address)

trans_ip.write(0x10, H_float_re.physical_address)
trans_ip.write(0x18, H_float_im.physical_address)
trans_ip.write(0x20, DDR_H_axi_re.physical_address)
trans_ip.write(0x28, DDR_H_axi_im.physical_address)
print('binded address!')

FISTA memory allocated!
TRANS memory allocated!
binded address!


In [ ]:
###############################################################

def trans_frame(img,dis_text='PSF'):
    dis_psf = cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)
    dis_psf = np.uint8(dis_psf)
    dis_psf = cv2.resize(dis_psf,(640,480))
    show_img = vdma.writechannel.newframe()
    cv2.cvtColor(dis_psf, cv2.COLOR_GRAY2RGB,dst=show_img)
    if dis_text is not '':
        cv2.putText(show_img,dis_text,(20,70),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    vdma.writechannel.writeframe(show_img)

def flash_frames(graph_name='opening',i_dis=0):
    dis_path = graphic_dir+graph_name+'_{}.jpg'.format(i_dis)
    result = cv2.imread(dis_path)
    result = cv2.resize(result,(640,480))
    show_img = vdma.writechannel.newframe()
    cv2.cvtColor(result, cv2.COLOR_BGR2RGB,dst=show_img)
    vdma.writechannel.writeframe(show_img)
def loaddata_hls(show_im=True):
    psf = Image.open(psfname).convert('RGB')
    psf = np.array(psf, dtype='float32')
    data = Image.open(imgname).convert('RGB')
    data = np.array(data, dtype='float32')

    bg = np.mean(0.114*psf[5:15,5:15,0]+0.587*psf[5:15,5:15,1]+0.2989*psf[5:15,5:15,2]) 
    bg = (bg-127.5)/127.5

    psf = cv2.resize(psf,(160,120))
    data = cv2.resize(data,(160,120))

    
    psf = (psf-127.5)/127.5
    data = (data-127.5)/127.5
   
    psf = 0.114*psf[:,:,0]+ 0.587*psf[:,:,1]+0.2989*psf[:,:,2]
    data = 0.114*data[:,:,0]+ 0.587*data[:,:,1]+0.2989*data[:,:,2]
    
    psf -= bg
    data -= bg

    psf /= np.linalg.norm(psf.ravel())
    data /= np.linalg.norm(data.ravel())
    
    if show_im:
        trans_frame(psf,'PSF')
        
        time.sleep(3)
        
        trans_frame(data,'RAW')

    return psf, data

def initMatrices(h):
    pixel_start = (np.max(h) + np.min(h))/2
    print("pixel start=",pixel_start)
    x = np.ones(h.shape)*pixel_start

    init_shape = h.shape
    padded_shape = [nextPow2(2*n - 1) for n in init_shape]
    starti = (padded_shape[0]- init_shape[0])//2
    endi = starti + init_shape[0]
    startj = (padded_shape[1]//2) - (init_shape[1]//2)
    endj = startj + init_shape[1]
    hpad = np.zeros(padded_shape)
    hpad[starti:endi, startj:endj] = h

    H = fft.fft2(fft.ifftshift(hpad), norm="ortho")
    Hadj = np.conj(H)

    def crop(X):
        return X[starti:endi, startj:endj]

    def pad(v):
        vpad = np.zeros(padded_shape).astype(np.complex64)
        vpad[starti:endi, startj:endj] = v
        return vpad

    utils = [crop, pad]
    v = np.real(pad(x))
    
    return H, Hadj, v, utils

def nextPow2(n):
    return int(2**np.ceil(np.log2(n)))

def fista_hls(vk, tk, xk, parent_var):
    H, Hadj, b, crop, pad, alpha, proj = parent_var
    #PS:     gradient = grad(Hadj, H, vk, b, crop, pad)
    ########with PL#############
    #DDR_H_axi_re,DDR_H_axi_im DONE
    start_t=time.time()
    np.copyto(DDR_vk_axi, np.array(vk))
    #vk_mid_axi_re,vk_mid_axi_im,DDR_grad_out DON'T NEED SET
    grad_ip.write(0x00, 1)
    x_k1 = xk
    t_k1 = (1+np.sqrt(1+4*tk**2))/2
    time.sleep(0.15)
    isready = grad_ip.read(0x00)
    while( isready == 1 ):
        isready = grad_ip.read(0x00)
    dur=time.time()-start_t
    vk -= alpha*DDR_grad_out
    xk = proj(vk)
    
    vk = xk+(tk-1)/t_k1*(xk - x_k1)
    tk = t_k1
   
    return vk, tk, xk,dur
###################################################################
def reconstruct(iter_host,use_admm=1):
    vdma.readchannel.stop()
    vdma.readchannel.start()
    vdma.writechannel.start()
    psf, data = loaddata_hls(True)
    #####################
    H, Hadj,vs, utils = initMatrices(psf)
    crop = utils[0]
    pad = utils[1]
    pixel_start = (np.max(psf) + np.min(psf))/2
    #print("pixel start=",pixel_start)
    if iter_host<=0:
        iter_host=1
    ##########################
    if(use_admm):
        admm_im,admm_xk = runADMM(psf, data,2)
        admm_im = cv2.normalize(admm_im,None,0,1,cv2.NORM_MINMAX)
        admm_xk = cv2.normalize(admm_xk,None,0,1,cv2.NORM_MINMAX)
        admm_im = admm_im*pixel_start
        admm_xk = admm_xk*pixel_start
        vk= np.real(pad(admm_im))
        xk = np.real(pad(admm_xk))
        #xk = vs
        trans_frame(admm_im,'ADMM init')
    else:
        xk = vs
        vk = vs
        
    alpha = np.real(1.8/(np.max(Hadj * H)))
    iterations = 0

    def non_neg(xi):
        xi[xi<0]=0
        return xi    
    #proj = lambda x:x #Do no projection
    proj = non_neg #Enforce nonnegativity at every gradient step. Comment out as needed.
    parent_var = [H, Hadj, data, crop, pad, alpha, proj]
     
    #### uncomment for FISTA update ################
    tk = 23.28346 #1
    #xk = v
    ################################################
    psf_H_re = H.real
    psf_H_im = H.imag
    np.copyto(H_float_re, np.array(psf_H_re))
    np.copyto(H_float_im, np.array(psf_H_im))
    trans_ip.write(0x00, 1)
    start_t=time.time()
    time.sleep(0.07)
    isready = trans_ip.read(0x00)
    while( isready == 1 ):
        isready = trans_ip.read(0x00)
    np.copyto(DDR_b_axi, np.array(data))
    #############################################
    #print("Initialization done!")
    overlay.axi_gpio_rgb[0].write(1)
    start_time=time.time()
    dur=0
    for iterations in range(iter_host):     
        # FISTA update
        vk, tk, xk,dur_r = fista_hls(vk, tk, xk, parent_var)
        dur=dur+dur_r
        if iterations % 1 == 0:
            image = proj(crop(vk))
            if  iterations>0:
                trans_frame(image,'iter{},grad time:{:.3f}'.format(iterations,dur/1))
                dur=0
    
    overlay.axi_gpio_rgb[0].write(0)
    end_time=time.time()
    dur_time=end_time-start_time
    print("dur_time:",dur_time,"each round:",dur_time/iter_host)
    image = proj(crop(vk))
    trans_frame(image,'finish {},press btn[1]'.format(iter_host))
    plt.imsave('result.jpg',image,cmap='gray')
#     start = time.time()
#     keydown=0
#     while(time.time()-start<30):
#         keydown=overlay.axi_gpio_btn4.read()
#         if(keydown == 2):
#             break
#         time.sleep(0.08)
#     return image    

def fast_admm(iter_admm):
    vdma.readchannel.stop()
    vdma.readchannel.start()
    vdma.writechannel.start()
    time.sleep(0.2)
    #############################
    psf = Image.open(psfname).convert('RGB')
    psf = np.array(psf, dtype='float32')

    bg = np.mean(0.114*psf[5:15,5:15,0]+0.587*psf[5:15,5:15,1]+0.2989*psf[5:15,5:15,2]) 
    bg = (bg-127.5)/127.5

    psf = cv2.resize(psf,(160,120))  
    psf = (psf-127.5)/127.5
    psf = 0.114*psf[:,:,0]+ 0.587*psf[:,:,1]+0.2989*psf[:,:,2]
    psf -= bg
    psf /= np.linalg.norm(psf.ravel())
    
    H_fft = precompute_H_fft(psf)
    
    X_divmat = precompute_X_divmat()
    PsiTPsi = precompute_PsiTPsi()
    R_divmat = precompute_R_divmat(H_fft, PsiTPsi)
    x=30
    y=30
    ##########################
    while(overlay.axi_gpio_sw0.read()):
        keydown=overlay.axi_gpio_btn4.read()
        if(keydown==8):
            list4[4]=x
            for i in list4:
                ser.write(chr(i).encode("utf-8"))
        elif(keydown==1):
            list5[4]=x
            for i in list5:
                ser.write(chr(i).encode("utf-8"))
        elif(keydown==4):
            list1[4]=y
            for i in list1:
                ser.write(chr(i).encode("utf-8"))
        elif(keydown==2):
            list2[4]=y
            for i in list2:
                ser.write(chr(i).encode("utf-8"))
        frame = vdma.readchannel.readframe()
        data = cv2.resize(frame,(160,120))
        data = np.array(data, dtype='float32')
        data = (data-127.5)/127.5
        data = 0.114*data[:,:,1]+ 0.587*data[:,:,2]+0.2989*data[:,:,0]
        data -= bg
        data /= np.linalg.norm(data.ravel())
        X,U,V,W,xi,eta,rho = init_Matrices(H_fft)
        for i in range(iter_admm):
            X,U,V,W,xi,eta,rho = ADMM_Step(X,U,V,W,xi,eta,rho, [H_fft, data, X_divmat, R_divmat])
        image = C(V)
        image[image<0] = 0
        trans_frame(image,'ADMM')
        
    r,g,b=cv2.split(frame)
    om=cv2.merge([b,r,g])        
    img_save_name = '/home/xilinx/jupyter_notebooks/demo1101/raw.jpg'
    dis_text = 'RAW captured!'
    tip_text = 'Press button[1] again to save.'
    font=cv2.FONT_HERSHEY_PLAIN#cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,dis_text,(20,70),font,3,(0,0,255),2)
    cv2.putText(frame,tip_text,(160,420),font,1,(0,0,255),1)
    vdma.writechannel.writeframe(frame)
    plt.imsave(img_save_name,om)
    start = time.time()
    keydown=0
    while(time.time()-start<30):
        keydown=overlay.axi_gpio_btn4.read()
        if(keydown == 2):
            break
        time.sleep(0.08)
    return   

In [ ]:
# Hyper-parameters in the ADMM implementation (like step size in GD)
mu1 = 1e-6
mu2 = 1e-5
mu3 = 4e-5
tau = 0.0001
sensor_size = np.array((120,160))
full_size = 2*sensor_size
def U_update(eta, image_est, tau):
    return SoftThresh(Psi(image_est) + eta/mu2, tau/mu2)

def SoftThresh(x, tau):
    # numpy automatically applies functions to each element of the array
    return np.sign(x)*np.maximum(0, np.abs(x) - tau)

def Psi(v):
    return np.stack((np.roll(v,1,axis=0) - v, np.roll(v, 1, axis=1) - v), axis=2)

def X_update(xi, image_est, H_fft, sensor_reading, X_divmat):
    return X_divmat * (xi + mu1*M(image_est, H_fft) + CT(sensor_reading))

def M(vk, H_fft):
    return np.real(fft.fftshift(fft.ifft2(fft.fft2(fft.ifftshift(vk))*H_fft)))

def C(M):
    # Image stored as matrix (row-column rather than x-y)
    top = (full_size[0] - sensor_size[0])//2
    bottom = (full_size[0] + sensor_size[0])//2
    left = (full_size[1] - sensor_size[1])//2
    right = (full_size[1] + sensor_size[1])//2
    return M[top:bottom,left:right]

def CT(b):
    v_pad = (full_size[0] - sensor_size[0])//2
    h_pad = (full_size[1] - sensor_size[1])//2
    return np.pad(b, ((v_pad, v_pad), (h_pad, h_pad)), 'constant',constant_values=(0,0))


def precompute_X_divmat(): 
    """Only call this function once! 
    Store it in a variable and only use that variable 
    during every update step"""
    return 1./(CT(np.ones(sensor_size)) + mu1)

def W_update(rho, image_est):
    return np.maximum(rho/mu3 + image_est, 0)

def r_calc(w, rho, u, eta, x, xi, H_fft):
    return (mu3*w - rho)+PsiT(mu2*u - eta) + MT(mu1*x - xi, H_fft)

def V_update(w, rho, u, eta, x, xi, H_fft, R_divmat):
    freq_space_result = R_divmat*fft.fft2( fft.ifftshift(r_calc(w, rho, u, eta, x, xi, H_fft)) )
    return np.real(fft.fftshift(fft.ifft2(freq_space_result)))

def PsiT(U):
    diff1 = np.roll(U[...,0],-1,axis=0) - U[...,0]
    diff2 = np.roll(U[...,1],-1,axis=1) - U[...,1]
    return diff1 + diff2

def MT(x, H_fft):
    x_zeroed = fft.ifftshift(x)
    return np.real(fft.fftshift(fft.ifft2(fft.fft2(x_zeroed) * np.conj(H_fft))))

def precompute_PsiTPsi():
    PsiTPsi = np.zeros(full_size)
    PsiTPsi[0,0] = 4
    PsiTPsi[0,1] = PsiTPsi[1,0] = PsiTPsi[0,-1] = PsiTPsi[-1,0] = -1
    PsiTPsi = fft.fft2(PsiTPsi)
    return PsiTPsi


def precompute_R_divmat(H_fft, PsiTPsi): 
    """Only call this function once! 
    Store it in a variable and only use that variable 
    during every update step"""
    MTM_component = mu1*(np.abs(np.conj(H_fft)*H_fft))
    PsiTPsi_component = mu2*np.abs(PsiTPsi)
    id_component = mu3
    """This matrix is a mask in frequency space. So we will only use
    it on images that have already been transformed via an fft"""
    return 1./(MTM_component + PsiTPsi_component + id_component)

def xi_update(xi, V, H_fft, X):
    return xi + mu1*(M(V,H_fft) - X)

def eta_update(eta, V, U):
    return eta + mu2*(Psi(V) - U)

def rho_update(rho, V, W):
    return rho + mu3*(V - W)


def init_Matrices(H_fft):
    X = np.zeros(full_size)
    U = np.zeros((full_size[0], full_size[1], 2))
    V = np.zeros(full_size)
    W = np.zeros(full_size)

    xi = np.zeros_like(M(V,H_fft))
    eta = np.zeros_like(Psi(V))
    rho = np.zeros_like(W)
    return X,U,V,W,xi,eta,rho


def precompute_H_fft(psf):
    return fft.fft2(fft.ifftshift(CT(psf)))

def ADMM_Step(X,U,V,W,xi,eta,rho, precomputed):
    H_fft, data, X_divmat, R_divmat = precomputed
    U = U_update(eta, V, tau)
    X = X_update(xi, V, H_fft, data, X_divmat)
    V = V_update(W, rho, U, eta, X, xi, H_fft, R_divmat)
    W = W_update(rho, V)
    xi = xi_update(xi, V, H_fft, X)
    eta = eta_update(eta, V, U)
    rho = rho_update(rho, V, W)
    
    return X,U,V,W,xi,eta,rho


def runADMM(psf, data,iters):
    H_fft = precompute_H_fft(psf)
    X,U,V,W,xi,eta,rho = init_Matrices(H_fft)
    X_divmat = precompute_X_divmat()
    PsiTPsi = precompute_PsiTPsi()
    R_divmat = precompute_R_divmat(H_fft, PsiTPsi)
    
    for i in range(iters):
        X,U,V,W,xi,eta,rho = ADMM_Step(X,U,V,W,xi,eta,rho, [H_fft, data, X_divmat, R_divmat])
        if i== iters-2:
            image_f = C(V)
    image = C(V)
    image[image<0] = 0
    return image,image_f

In [ ]:
##################### usb-serial #####################
port_list = list(serial.tools.list_ports.comports())

if len(port_list) == 0:
    print('无可用串口')
else:
    for i in range(0,len(port_list)):
        print(port_list[i])

    list1=[0x55,0x55,0x04,0x01,0x01,0x00]
    list2=[0x55,0x55,0x04,0x02,0x01,0x00]
    list4=[0x55,0x55,0x04,0x04,0x01,0x00]
    list5=[0x55,0x55,0x04,0x05,0x01,0x00]
    list7=[0x55,0x55,0x02,0x07]

    list_one=[0x55,0x55,0x08,0x03,0x01,0x00,0x04,0x01,0xdc,0x05]

    portx='/dev/ttyUSB0'
    bps=9600
    #超时设置,None：永远等待操作，0为立即返回请求结果，其他值为等待超时时间(单位为秒）
    timex=1
    ser=serial.Serial(portx,bps,timeout=timex)

In [ ]:
overlay.axi_gpio_rgb[3].write(0)#sign as setting up
###########################MAIN##########################################################
vdma.readchannel.start()
vdma.writechannel.start()
keydown=0
flag_stream = 0
flag_capture = 0
flag_re = 0

i_dis=0
while True:
    keydown=overlay.axi_gpio_btn4.read()
    if(keydown==1):#ON
        overlay.axi_gpio_led4[0].write(1)
        blink_led4(1)
        flag_stream = 0
        flag_capture = 0
        flag_re = 0
        break
    #######Display Opening Images on Screen#########
    if(i_dis>2):
        i_dis=0
    flash_frames('opening',i_dis)
    i_dis=i_dis+1
    time.sleep(0.08)
    
for led in range(0,4):
    overlay.axi_gpio_led4[led].write(0)
####################################################################################
while True:
    mode = overlay.axi_gpio_sw0.read()
    keydown=overlay.axi_gpio_btn4.read()
    if(keydown==1):#OFF(EXIT)
        overlay.axi_gpio_rgb[5].write(0)
        overlay.axi_gpio_led4[0].write(1)
        blink_led4(3)
        flag_stream = 0
        flag_capture = 0
        flag_re = 0
        break
    elif(keydown==8):#AUTO-TRACE
        if(mode==0):
            overlay.axi_gpio_rgb[5].write(0)
            overlay.axi_gpio_led4[3].write(1)
            auto_trace()
            water_led4(2)
            flag_stream = 0
            flag_capture = 0
            flag_re = 0
        else:
            overlay.axi_gpio_rgb[4].write(0)
            overlay.axi_gpio_led4[3].write(1)
            manual_arm()
            water_led4(2)
            flag_stream = 0
            flag_capture = 0
            flag_re = 0
    elif(keydown==4):#CAPTURE
        if(flag_capture == 0):
            overlay.axi_gpio_rgb[5].write(0)
            flag_capture = 1
            overlay.axi_gpio_led4[2].write(1)
            capture_raw(mode)
            water_led4(2)
            flag_stream = 0
            flag_re = 0
    elif(keydown==2):#RECONSTRUCT
        if(flag_re == 0):
            if(mode==0):
                overlay.axi_gpio_rgb[5].write(0)
                flag_re = 1
                overlay.axi_gpio_led4[1].write(1)
                reconstruct(45,1)
                flag_stream = 0
                flag_capture = 0
            else:
                overlay.axi_gpio_rgb[5].write(0)
                flag_re = 1
                overlay.axi_gpio_led4[1].write(1)
                fast_admm(1)
                flag_stream = 0
                flag_capture = 0
    else:
        if(flag_stream==0):
            overlay.axi_gpio_rgb[5].write(1)
            flag_stream=1
            flag_capture = 0
            flag_re = 0
            for led in range(0,4):
                overlay.axi_gpio_led4[led].write(0)
            recover_stream()
    time.sleep(0.08)
print('Exit!')



keydown=0
i_dis=0
while True:
    keydown=overlay.axi_gpio_btn4.read()
    if(keydown==1):#ON
        overlay.axi_gpio_led4[0].write(1)
        blink_led4(1)
        break
    if(i_dis>1):
        i_dis=0
    flash_frames('ending',i_dis)
    i_dis=i_dis+1
    time.sleep(0.08)
for led in range(0,4):
    overlay.axi_gpio_led4[led].write(0)
#######################Reset & Clean Up##################
vdma.readchannel.stop()
vdma.writechannel.stop()
vdma.readchannel.reset()
vdma.writechannel.reset()
### clean up xlink#####
xlnk.xlnk_reset()
print("---------------")  

In [ ]:
while(1):
    time.sleep(5)
    overlay.axi_gpio_rgb[5].write(0)
    overlay.axi_gpio_led4[1].write(1)
    reconstruct(45,1)

In [8]:
ser.close()#关闭串口  